In [ ]:
# %pip install pinecone-client
# %pip install python-dotenv
# %pip install openai
# %pip install chromadb-client

# %pip install llama-index-vector-stores-chroma

In [ ]:
from openai import OpenAI, AzureOpenAI
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

In [ ]:
from pinecone import Pinecone

# pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
# index = pc.Index("test-pdf")

In [ ]:
# from openai import OpenAI
# client = OpenAI()

# response = client.embeddings.create(
#     input="Your text string goes here",
#     model="text-embedding-3-large"
# )

# print(response.data[0].embedding)

In [ ]:
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = "2024-02-01",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

response = client.embeddings.create(
    input="Your text string goes here",
    model="ada_gcal"
)

print(response.data[0].embedding)

In [ ]:
def generate_embeddings(text, model="ada_gcal"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# df_bills['ada_v2'] = df_bills["text"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002'))

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("./pdf/grokking.pdf")
pages = loader.load_and_split()

In [ ]:
print(len(pages))
print()

full_document = ""

for page in pages:
  full_document += page.page_content

print(full_document)

In [ ]:
import re

# first, extract Section Headers
pattern = r"\n\d+ [A-Z][a-zA-Z\s:]+\n"

# Find all matches
matches = re.findall(pattern, full_document)

matches.insert(0, "Abstract")

# Print all section titles
for match in matches:
    print(match.strip())
    print()

In [ ]:
pattern = r"\n\d+ [A-Z][a-zA-Z\s:]+\n"

# Split the text into sections
sections = re.split(pattern, full_document)

# number of sections should match with number of section titles (aka matches of regex pattern)
# print(sections[0])
# print(sections[2])
# print(len(sections))

# Create a dictionary to store section titles and contents
section_contents = {}

# Use zip to iterate over matches and sections simultaneously
for match, section in zip(matches, sections):
    section_title = match.strip()
    content = section.strip()
    section_contents[section_title] = content

section_contents.keys()


In [ ]:
section_contents["6 Conclusion"] #TODO: process this to ONLY the abstract content

In [ ]:
for title, content in section_contents.items():
    print(title)
    print(len(content))
    print()

In [ ]:
subsection = section_contents["3 Why Generalization Occurs: Representations and Dynamics"]
subsection

In [ ]:
# then extract subsections from sections
pattern = r"\n\d+\.\d+ [A-Z][a-zA-Z\s:]+\n"

subsections = re.findall(pattern, subsection)
print(len(subsections))
for match in subsections:
    print(match.strip())
    print()

In [105]:
# extract references from conclusion
abstract = section_contents["Abstract"]
section_contents["Abstract"] = abstract.split("Abstract")[1]

section_contents["Abstract"]

'\nWe aim to understand grokking , a phenomenon where models generalize long after\noverﬁtting their training set. We present both a microscopic analysis anchored by an\neffective theory and a macroscopic analysis of phase diagrams describing learning\nperformance across hyperparameters. We ﬁnd that generalization originates from\nstructured representations whose training dynamics and dependence on training set\nsize can be predicted by our effective theory in a toy setting. We observe empirically\nthe presence of four learning phases: comprehension ,grokking ,memorization , and\nconfusion . We ﬁnd representation learning to occur only in a “Goldilocks zone”\n(including comprehension and grokking) between memorization and confusion.\nWe ﬁnd on transformers the grokking phase stays closer to the memorization phase\n(compared to the comprehension phase), leading to delayed generalization. The\nGoldilocks phase is reminiscent of “intelligence from starvation” in Darwinian\nevolution, wher

In [ ]:
# extract references from conclusion
conclusion = section_contents["6 Conclusion"]
section_contents["6 Conclusion"] = conclusion.split("References")[0]
# conclusion.split("References")[1]

section_contents["6 Conclusion"]

In [ ]:
# run subsection splitting algo

for title, section in section_contents.items():
  pattern = r"\n\d+\.\d+ [A-Z][a-zA-Z\s:]+\n"
  subsections_headers = re.findall(pattern, section)
  
  if len(subsections_headers) > 0:
    subsection_map = {}
    subsections_content = re.split(pattern, section)

    subsections_headers.insert(0, title)
    for subsection_header, subsection in zip(subsections_headers, subsections_content):
      subsection_title = subsection_header.strip()
      content = subsection.strip()
      subsection_map[subsection_title] = content

    print(subsection_map.keys())
    section_contents[title] = subsection_map
    

In [ ]:
len(section_contents["3 Why Generalization Occurs: Representations and Dynamics"])
len(section_contents["4 Delayed Generalization: A Phase Diagram"])

In [ ]:
# use this for section title-based splitting
section_contents["4 Delayed Generalization: A Phase Diagram"]['4.1 Phase diagram of a toy model']

In [ ]:
def generate_embeddings(text, model="ada_gcal"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# df_bills['ada_v2'] = df_bills["text"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002'))

In [92]:
import chromadb.utils.embedding_functions as embedding_functions
azure_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
                api_base=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_type="azure",
                api_version = "2024-02-01",
                model_name="ada_gcal"
            )

In [89]:
import chromadb
# Example setup of the client to connect to your chroma server
client = chromadb.HttpClient(host='localhost', port=8000)

In [98]:
# client.delete_collection(name="test_pdf") 
# Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible

In [100]:
collection = client.create_collection(
        embedding_function=azure_ef,
        name="test_pdf",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )
collection = client.get_collection(name="test_pdf", embedding_function=azure_ef)

In [112]:
collection.peek() # returns a list of the first 10 items in the collection
# collection.count() # returns the number of items in the collection
# collection.modify(name="new_name") # Rename the collection

{'ids': ['143fd07b-a5ad-4668-a086-010b0e10f590',
  '5b661e55-62a4-4492-9e02-c9d59e187eb5',
  '690f199b-6cd8-465b-818d-e4433c76730b',
  'abe277df-847a-4ace-a50f-6aeb98700b7b',
  'dd082b0d-8545-4644-914d-5ac30eb93e9f'],
 'embeddings': [[-0.02222548983991146,
   -0.0005509211914613843,
   0.04491744190454483,
   -0.01904258131980896,
   -0.009329218417406082,
   -0.001376231200993061,
   0.01250526774674654,
   -0.012532707303762436,
   -0.013966388069093227,
   -0.048731446266174316,
   0.02083982713520527,
   0.032789457589387894,
   -0.008183645084500313,
   0.007895536720752716,
   -0.014377971179783344,
   0.006897447630763054,
   0.03424371778964996,
   0.019152335822582245,
   -0.0026324172504246235,
   -0.016024304553866386,
   -0.01476211566478014,
   0.03665833920240402,
   -0.0021762459073215723,
   -0.0278916172683239,
   -0.0037762753199785948,
   0.005604390520602465,
   0.01585967093706131,
   -0.030896175652742386,
   -0.018095938488841057,
   0.01978342980146408,
   0.020

In [116]:
# collection.add(
#     documents=["lorem ipsum...", "doc2", "doc3", ...],
#     metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
#     ids=["id1", "id2", "id3", ...]
# )

import uuid

for title, content in section_contents.items():
  unique_id = str(uuid.uuid4())
  if type(content) == dict:
    for t, c in content.items():
      collection.add(
      documents = [c],
      metadatas=[{"section_title": t}],
      ids=[unique_id]
    )
  else:
    collection.add(
      documents = [content],
      metadatas=[{"section_title": title}],
      ids=[unique_id]
    )

In [131]:
multiQueryPrompt = """You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines only.

    For example:
    User question: "What is the conclusion of the paper?"

    Generated questions:
    What is the main takeaway from the paper?
    What are the key findings of the paper?
    What is the summary of the paper?
    What is the final thought of the paper?
    What is the ending of the paper?
    
    Output format should be as follows:

    'What is Bill Gates known for?'
│   "Can you provide information about Bill Gates' background?"

    And not this format:

    '1. What is Bill Gates known for?'
│   "2. Can you provide information about Bill Gates' background?"
    """

In [133]:
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = "2024-02-01",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

response = client.chat.completions.create(
    model="cursor-gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": multiQueryPrompt},
        {"role": "user", "content": "why does generalization occur?"}
    ]
)

#print(response)
print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)

questions = response.choices[0].message.content.split("\n")
print(questions)

{
  "id": "chatcmpl-9ExQpA2wfug1eMh3Bn29OwZhdmQkO",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "What are the reasons for the occurrence of generalization?\nWhat causes generalization to happen?\nWhat factors lead to the phenomenon of generalization?\nHow does the process of generalization take place?\nCan you explain why generalization takes place?",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1713352075,
  "model": "gpt-4-32k",
  

In [134]:
relevant_docs = collection.query(
    query_texts=questions,
    n_results=3,
    # where={"metadata_field": "is_equal_to_this"},
    # where_document={"$contains":"search_string"}
)

{'ids': [['690f199b-6cd8-465b-818d-e4433c76730b',
   'abe277df-847a-4ace-a50f-6aeb98700b7b',
   '6073b650-1c5e-464e-be83-d1057cc057ab'],
  ['690f199b-6cd8-465b-818d-e4433c76730b',
   'abe277df-847a-4ace-a50f-6aeb98700b7b',
   '6073b650-1c5e-464e-be83-d1057cc057ab'],
  ['690f199b-6cd8-465b-818d-e4433c76730b',
   '6073b650-1c5e-464e-be83-d1057cc057ab',
   'abe277df-847a-4ace-a50f-6aeb98700b7b'],
  ['690f199b-6cd8-465b-818d-e4433c76730b',
   'abe277df-847a-4ace-a50f-6aeb98700b7b',
   '6073b650-1c5e-464e-be83-d1057cc057ab'],
  ['690f199b-6cd8-465b-818d-e4433c76730b',
   'abe277df-847a-4ace-a50f-6aeb98700b7b',
   '6073b650-1c5e-464e-be83-d1057cc057ab']],
 'distances': [[0.16739195667186746, 0.19371908438399088, 0.19603750730815583],
  [0.17163321829125977, 0.1989038123081921, 0.19950455764466646],
  [0.1685012392923153, 0.1841849578789404, 0.18563202714786886],
  [0.1632188509917324, 0.18209705867508197, 0.189131988730474],
  [0.15290203061985708, 0.1752307672854816, 0.17865061200590482]],


In [141]:
# qaPrompt = """Here is the question you need to answer:

#     \n --- \n {query_str} \n --- \n

#     Here is any available background question + answer pairs:

#     \n --- \n {q_a_pairs} \n --- \n

#     Here is additional context relevant to the question:

#     \n --- \n {context_str} \n --- \n

#     Use the above context and any background question + answer pairs to answer the question: \n {query_str}
#     """

# TODO: q_a_pairs from FT-ed model

qaPrompt = """
Answer the question based on the context below. If the
question cannot be answered using the additional context, answer
with "I cannot answer the question based on the context provided".

Here is the question you need to answer:

    ###{query_str}###

Here is additional context relevant to the question:

    ###{context_str}###

Use the above context ONLY to answer the question: \n {query_str}

Do not create new context or use external sources to answer the question.
    """

prompt_template = PromptTemplate(input_variables=["query_str", "context_str"], template=qaPrompt)

In [ ]:
query = "What libraries can be used to access Large Language Models?"
formatted_qa_prompt = prompt_template.format(query_str=query, context_str="context 1")
print(formatted_qa_prompt)

response = client.chat.completions.create(
    model="cursor-gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": multiQueryPrompt},
        {"role": "user", "content": "why does generalization occur?"}
    ]
)

#print(response)
print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)

questions = response.choices[0].message.content.split("\n")
print(questions)